### This script helps you pull data directly from PHE Fingertips

In [1]:
# Install the Fingertips package for python (YOU ONLY NEED TO DO THIS ONCE)
# Delete this cell or comment the code below (e.g. #...) after installing and before ever running this code again

#!pip install fingertips_py

In [2]:
# Import Fingertips and python's main preprocessing package pandas, as well as the datetime package
# Documentation for the Fingertips package can be found here: https://fingertips-py.readthedocs.io/en/latest/#indices-and-tables

import pandas as pd
from datetime import datetime
import fingertips_py as ftp

In [3]:
# Save all Fingertips indicators as a lookup df

indicators = ftp.metadata.get_metadata_for_all_indicators() #Use a specific function to save all metadata for all indicators from Fingertips
indicators["IndicatorName"] = indicators["Descriptive"].apply(lambda x: x["Name"]) #The original column is populated with a dictionary in each row - this function returns only the 'Name' value from that dictionary
indicators["IndicatorNameLower"] = indicators["IndicatorName"].str.lower() #Change indicator names to lowercase
indicators["Source"] = indicators["Descriptive"].apply(lambda x: x["DataSource"]) #Same as above, but returns the DataSource value from the dictionary
indicators["YearType"] = indicators["YearType"].apply(lambda x: x["Name"]) #Similar to above
indicators["ValueType"] = indicators["ValueType"].apply(lambda x: x["Name"])  #Similar to above
indicators["UnitLabel"] = indicators["Unit"].apply(lambda x: x["Label"]) #Similar to above
indicators["Unit"] = indicators["Unit"].apply(lambda x: x["Value"]) #Similar to above
indicators = indicators[["IID", "IndicatorName", "IndicatorNameLower", "Source", "YearType", "ValueType", "UnitLabel", "Unit"]] #Select only helpful columns
indicators.head() #Preview the dataframe

,IID,IndicatorName,IndicatorNameLower,Source,YearType,ValueType,UnitLabel,Unit
108,108,Under 75 mortality rate from all causes,under 75 mortality rate from all causes,Office for Health Improvement and Disparities ...,Calendar,Directly standardised rate,"per 100,000",100000.0
113,113,Smoking attributable mortality (Old Method),smoking attributable mortality (old method),"ONS mortality file, ONS LSOA single year of ag...",Calendar,Directly standardised rate,"per 100,000",100000.0
114,114,QOF Total List Size,qof total list size,"Quality and Outcomes Framework (QOF), NHS Digital",Financial,Count,,1.0
200,200,Learning disability: QOF prevalence,learning disability: qof prevalence,"Quality and Outcomes Framework (QOF), NHS Digital",Financial,Proportion,%,100.0
212,212,Stroke: QOF prevalence (all ages),stroke: qof prevalence (all ages),"Quality and Outcomes Framework (QOF), NHS Digital",Financial,Proportion,%,100.0


In [4]:
# Create is a helper function to find indicators based on a simple word search
# This returns indicators whose names match certain strings - note that the word search is case sensitive
# This cell doesn't return anything

def find_indicator(text_to_find, just_names=True): #just_names returns the full indicator names that match the search (rather than a dataframe), which can be useful as they are often cut off if you are returning the whole dataframe
    
    inds = [i for i in indicators["IndicatorNameLower"].unique() if text_to_find.lower() in i] #Return all indicator names that contain the search term (make lowercase to increase chances of finding a match)
    if len(inds) == 0: #If no indicators matched, alert the user and ask them to try again
        print("\n****\nNo indicators matched your text. Please re-run the cell with and use a different search term.\n****\n")
        return None
    inds = indicators.loc[indicators.IndicatorNameLower.isin(inds)].copy() #Take a subset of the main indicators df from the cell above that only includes the matching indicators
    if just_names: #If the just_names argument is True, return a list of the indicator names (rather than a subset of the main df)
        inds = inds.IndicatorName.unique()
    return inds #Returns either a subsetted dataframe from the 'indicators' dataframe or a list of full indicator names

In [5]:
# Allow a user to input a search term to search for matching indicators

indicator_word_search = input("Please type a phrase/word/partial word to search for indicators on Fingertips: ") #This saves a user input for the word search
indicator_search_results = find_indicator(indicator_word_search, just_names=True) #Use the user input to pull all indicator names that contain that string
if indicator_search_results is not None:
    print("\nIndicator names that match your word search:\n") #Print all indicators that match the user input
    for i in indicator_search_results:
        print(i) #Print all matching indicators one by one

Please type a phrase/word/partial word to search for indicators on Fingertips: decay

Indicator names that match your word search:

Tooth extractions due to decay for children admitted as inpatients
Children with one or more decayed,  missing or filled teeth
Proportion of five year old children free from dental decay
Percentage of three year olds with experience of visually obvious tooth decay
dmft (decayed, missing or filled teeth) in five year olds
Proportion of twelve year olds free from dental decay
Percentage of 5 year olds with experience of visually obvious dental decay


In [12]:
# Allow the user to select all or specific indicators

select_all_or_specific = input("Would you like to select all matching indicators above or list specific indicators? Please answer 'All' or 'Specific': ") #Allow a user to select all indicators listed in the output from the cell above or specific indicators

selected_indicators = [] #Create an empty list of indicators, which will be populated by user-selected indicators (if they don't select all)

def save_specific_list_of_indicators(): 
    #The functions asks a user to input their chosen indicators one at a time
    
    global selected_indicators #Call in local variables so they can be accessed within the function
    global list_of_inds_complete
    
    first_indicator_question = "Please paste the full indicator name that you would like to include (you will have an opportunity to add more than one): " #Set the instruction text for the user input box - text will vary whether it is the first or 2+ indicator
    subsequent_indicator_question = "Please paste the full indicator name that you would like to include: "
    
    if len(selected_indicators) == 0: #If the user hasn't selected their first indicator yet, uset the first question
        indicator = input(first_indicator_question) 
    else:
        indicator = input(subsequent_indicator_question) #Use the second question for subsequent indicators

    check_indicator = indicator in indicators.IndicatorName.unique() #Check that the indicator name provided is in the Fingertips indicators dataframe
    if check_indicator: #If the output of check_indicator is True, then add it to the list of selected indicators
        selected_indicators.append(indicator) 
    else: #If the indicator name provided by the user does not match any in the indicators df, then warn the user and give them a chance to re-enter the indicator name by using recursion (i.e. starting the function again)
        print("\n****\nIndicator name not found. Please make sure you paste the whole indicator name (note it is case sensitive).\n****")
        save_specific_list_of_indicators() #Use recursion to start the function again
        
    add_another = input("Would you like to add another indicator? Please answer with 'Yes' or 'No': ") #Ask the user if they want to add another indicator
    if add_another.lower() == "yes": #If yes, use recursion to call the function again
        save_specific_list_of_indicators()
    elif add_another.lower() == "no":
        return None
    

if select_all_or_specific.lower() == "all": #If the user wants to select all indicators from the output in the cell above...
    indicator_selection = indicators.loc[indicators.IndicatorName.isin(indicator_search_results)] #Make a subset of the Fingertips indicators df that matches the users' selected indicators
else: #Otherwise...
    save_specific_list_of_indicators() #Call the function above, which will repeat until the user has finished selecting all of their indicators
    indicator_selection = indicators.loc[indicators.IndicatorName.isin(selected_indicators)] #Take a subset of the Fingertips indicators df that matches the users selected indicators
print("\nInformation about your selected indicators is below:")
indicator_selection #Return metadata for the selected indicators

Would you like to select all matching indicators above or list specific indicators? Please answer 'All' or 'Specific': specific
Please paste the full indicator name that you would like to include (you will have an opportunity to add more than one): Children with one or more decayed,  missing or filled teeth
Would you like to add another indicator? Please answer with 'Yes' or 'No': yes
Please paste the full indicator name that you would like to include: Proportion of five year old children free from dental decay
Would you like to add another indicator? Please answer with 'Yes' or 'No': no

Information about your selected indicators is below:


,IID,IndicatorName,IndicatorNameLower,Source,YearType,ValueType,UnitLabel,Unit
90820,90820,"Children with one or more decayed, missing or...","children with one or more decayed, missing or...",National Dental Epidemiology Programme for Eng...,Academic,Proportion,%,100.0
92441,92441,Proportion of five year old children free from...,proportion of five year old children free from...,Dental Public Health Epidemiology Programme fo...,Academic,Proportion,%,100.0


In [13]:
# Pull data for chosen indicators and select geography / authority level
# Note that the large size of the dataframes means this cell can take a while to run

final_output = pd.DataFrame() #Create an empty dataframe so the output of each selected indicator can be saved in the same df

def pull_data_for_geography(df):
    #This function asks users to input a geography by which to subset the data
    
    chosen_geo = input("Please select your geography of choice by pasting one geography name from above: ") #User input is saved as a variable
    print("\n") #For clarity, add a blank line
    try: #Try and subset the data by the chosen geography
        df = df.loc[data["Area Type"] == chosen_geo]
    except KeyError: #If that geography doesn't exist in the data...
        print("Geography not found. Please make sure the geography is spelt correctly.") #Warn the user
        pull_data_for_geography(df) #Use recursion to re-call the function
    return df

for ind_ref in indicator_selection.IID.unique(): #Iterate through each chosen indicator
    data = ftp.retrieve_data.get_data_for_indicator_at_all_available_geographies(ind_ref) #Pull data for all geographies from Fingertips
    if data.shape[0] == 0:
        ind_name = indicator_selection.loc[indicator_selection.IID == ind_ref, "IndicatorName"].unique()[0]
        print("Indicator name:", ind_name)
        print("There is no data for this indicator.\n")
        continue
    geographies = data["Area Type"].unique() #Create a list of all unique geographies present in the data
    print("Indicator name:", data["Indicator Name"].unique()[0]) #Print the indicator name
    print("Data for this indicator is available at the following geographies (England will be included at all geographies):")
    for geo in geographies:
        print("- {}".format(geo)) #Print the geographies one at a time
    select_geo = input("\nFor this indicator, would you like to save data at all geography levels or select one geography? Please answer with 'All' or 'Specific': ") #User input for whether they want to select all geographies listed or just one
    if select_geo.lower() != "all": #If the user hasn't stated 'all', call the function above
        data = pull_data_for_geography(data)
    final_output = final_output.append(data, ignore_index=True) #Append the output to the empty df above

print("Output preview (includes all selected indicators):")
final_output.head() #Preview the output

C:\Users\Kieren.gera\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'fingertips.phe.org.uk'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Indicator name: Children with one or more decayed,  missing or filled teeth
Data for this indicator is available at the following geographies (England will be included at all geographies):
- England
- Counties & UAs (2020/21)
- CCGs (2020/21)
- Region

For this indicator, would you like to save data at all geography levels or select one geography? Please answer with 'All' or 'Specific': specific
Please select your geography of choice by pasting one geography name from above: England


Indicator name: Proportion of five year old children free from dental decay
There is no data for this indicator.

Output preview (includes all selected indicators):


,Indicator ID,Indicator Name,Parent Code,Parent Name,Area Code,Area Name,Area Type,Sex,Age,Category Type,...,Upper CI 99.8 limit,Count,Denominator,Value note,Recent Trend,Compared to England value or percentiles,Compared to percentiles,Time period Sortable,New data,Compared to goal
0,90820,"Children with one or more decayed, missing or...",NaN,NaN,E92000001,England,England,Persons,5 yrs,NaN,...,NaN,NaN,NaN,NaN,NaN,Not compared,Not compared,20110000,NaN,NaN
1,90820,"Children with one or more decayed, missing or...",E92000001,England,E92000001,England,England,Persons,5 yrs,NaN,...,NaN,NaN,NaN,NaN,NaN,Similar,Not compared,20110000,NaN,NaN
2,90820,"Children with one or more decayed, missing or...",NaN,NaN,E92000001,England,England,Persons,5 yrs,NaN,...,NaN,NaN,NaN,NaN,NaN,Not compared,Not compared,20140000,NaN,NaN
3,90820,"Children with one or more decayed, missing or...",E92000001,England,E92000001,England,England,Persons,5 yrs,NaN,...,NaN,NaN,NaN,NaN,NaN,Similar,Not compared,20140000,NaN,NaN
4,90820,"Children with one or more decayed, missing or...",NaN,NaN,E92000001,England,England,Persons,5 yrs,NaN,...,NaN,NaN,NaN,NaN,Cannot be calculated,Not compared,Not compared,20160000,NaN,NaN


In [8]:
# Save results

save_results = input("Would you like to save results as a csv? (If you have selected more than one indicator, this would save one csv per indicator.)")
if save_results.lower() == "yes":
    month_year = "_{0}_{1}".format(datetime.now().month, datetime.now().year) #Pull today's month and year to save as part of the filename (for version control)

    for indicator in final_output["Indicator Name"].unique(): #Loop through the selected indicators
        indicator_df = final_output.loc[final_output["Indicator Name"] == indicator] #Subset the final_output df to that indicator
        filename = indicator.replace(" ", "_").lower()[:50]
        filename = filename.replace(":", "") + month_year #Replace spaces in the indicator name with underscores
        indicator_df.to_csv("Fingertips outputs/{}.csv".format(filename), index=False) #Save the output in the Fingertips outputs folder

Would you like to save results as a csv? (If you have selected more than one indicator, this would save one csv per indicator.)yes



### Some DIY Fingertips functions for quick reference

- ftp.metadata.get_area_types_for_profile(profile_name) #Pull the area type IDs for a given profile
- ftp.retrieve_data.get_data_for_indicator_at_all_available_geographies(ind_ref) #Pull all data for an indicator
- ftp.retrieve_data.get_data_by_indicator_ids(ind_ref, geo_ref) #Pull data for a specific indicator and geography
- ftp.metadata.get_area_types_as_dict() #Presents a dictionary of area type IDs and names
- ftp.retrieve_data.get_all_areas_for_all_indicators() #Get all geography ID codes for all indicators
- ftp.metadata.get_area_type_ids_for_profile(profile_id) #Get geography ID codes for a specific profile on Fingertips